In [1]:
import os
import pandas as pd
import pymongo
os.chdir(r'C:\Users\bjros\OneDrive\Desktop\KU_Data_Analytics_Boot_Camp\Projects\Project_2\citi-service')

In [2]:
#GET DATA
citi_df = pd.read_csv("Resources/311_Call_Center_Service_Requests.csv")
citi_df_mod = citi_df[['SOURCE','DEPARTMENT','CATEGORY', 'TYPE', 'DETAIL', 'CREATION YEAR','CREATION DATE','CREATION TIME','STATUS','DAYS TO CLOSE', 'STREET ADDRESS', 'ZIP CODE', 'COUNCIL DISTRICT','LATITUDE', 'LONGITUDE']]

C:\Users\bjros\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#CLEAN DATA
# 1) Rename Columns & convert year to string
citi_df_mod.columns = ['source', 'department', 'category', 'type', 'detail','year','creation_date', 'creation_time','status', 'days_to_close', 'street_address', 'zip code', 'council_district','lat','lon']
citi_df_mod['year'] = citi_df_mod['year'].astype(str)

C:\Users\bjros\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [4]:
# 2) Add a month-date column 

listy = []
#Loop through the 'CREATION DATE' data
for i in range(len(citi_df_mod['creation_date'])):
    #Use Split Function to Pull Out the Month & Day Values for Each Record
    months = citi_df_mod['creation_date'][i].split("/")[0]
    days = citi_df_mod['creation_date'][i].split("/")[1]
    #Concatenate the Values
    month_days = str(months) + "/" + str(days)
    #Append Month/Day Value to List
    listy.append(month_days)
#Create a New Series to Unpack the Month/Day Values
citi_df_mod['creation_month-day'] = pd.Series(listy)
citi_df_mod['creation_month-day']

C:\Users\bjros\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0          12/18
1          01/12
2          11/15
3          11/16
4          11/27
           ...  
1467979    09/25
1467980    09/27
1467981    09/26
1467982    09/24
1467983    09/27
Name: creation_month-day, Length: 1467984, dtype: object

In [5]:
# 3) Fill in NaN
citi_df_mod = citi_df_mod.dropna(how="any")
citi_df_mod.dtypes

source                 object
department             object
category               object
type                   object
detail                 object
year                   object
creation_date          object
creation_time          object
status                 object
days_to_close          object
street_address         object
zip code              float64
council_district      float64
lat                   float64
lon                   float64
creation_month-day     object
dtype: object

In [6]:
# 4) Filter by Source
citi_df_mod = citi_df_mod.loc[(citi_df_mod["source"] == "WEB") | (citi_df_mod["source"] == "PHONE") |(citi_df_mod["source"] == "EMAIL") |(citi_df_mod["source"] == "WALK") | (citi_df_mod["source"] == "MAIL") ]
citi_df_mod

,source,department,category,type,detail,year,creation_date,creation_time,status,days_to_close,street_address,zip code,council_district,lat,lon,creation_month-day
0,PHONE,NHS,Trash / Recycling,Trash Collection,Missed by City,2019,12/18/2019,01:52 PM,RESOL,6,2822 E 9th St,64124.0,3.0,39.102651,-94.546809,12/18
1,WEB,City Managers Office,Streets / Roadways / Alleys,Snow / Ice,Snow Route Verification,2020,01/12/2020,03:37 PM,RESOL,5,5635 N Manchester Ave,64119.0,1.0,39.196584,-94.487959,01/12
2,PHONE,Public Works,Signs,Traffic Sign,New Sign Request,2019,11/15/2019,03:01 PM,RESOL,3,301 W 45th St,64111.0,4.0,39.045500,-94.590528,11/15
3,PHONE,Water Services,Storm Water / Sewer,Storm Water Referral,Repair,2019,11/16/2019,01:16 PM,RESOL,26,1001 W 55th St,64113.0,4.0,39.027698,-94.599931,11/16
4,WEB,Water Services,Water,Leak,In Street,2019,11/27/2019,04:01 PM,RESOL,7,5200 Cherry St,64110.0,4.0,39.031266,-94.581718,11/27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467978,PHONE,NHS,Trash / Recycling,Recycling,Missed by City,2019,09/25/2019,04:47 PM,RESOL,1,3939 INDIANA AVE,64130.0,3.0,39.053584,-94.544100,09/25
1467979,EMAIL,City Planning and Development,Property / Buildings / Construction,Construction Issue/Concern,Other,2019,09/25/2019,06:17 PM,RESOL,6,11111 N Wallace Ave,64157.0,1.0,39.296232,-94.478623,09/25
1467981,PHONE,Parks and Rec,Trees,Services,Plant City Tree Request,2019,09/26/2019,08:05 AM,RESOL,0,3614 Norledge Ave,64123.0,4.0,39.116513,-94.537826,09/26
1467982,PHONE,NHS,Mowing / Weeds,Public Property,Land Bank,2019,09/24/2019,01:19 PM,RESOL,104,5109 Prospect Ave,64130.0,5.0,39.033313,-94.554381,09/24


In [7]:
# 5) Rename values in source column
citi_df_mod["source"] = citi_df_mod["source"].str.replace("WEB","Online")
citi_df_mod["source"] = citi_df_mod["source"].str.replace("PHONE","Phone")
citi_df_mod["source"] = citi_df_mod["source"].str.replace("EMAIL","Email")
citi_df_mod["source"] = citi_df_mod["source"].str.replace("MAIL","Mail")
citi_df_mod["source"] = citi_df_mod["source"].str.replace("WALK","Walk-in")

# 6) Rename values in status column
citi_df_mod["status"] = citi_df_mod["status"].str.replace("OPEN","Open")
citi_df_mod["status"] = citi_df_mod["status"].str.replace("RESOL","Resolved")
citi_df_mod["status"] = citi_df_mod["status"].str.replace("DUP","Duplicate")
citi_df_mod["status"] = citi_df_mod["status"].str.replace("ASSIG","Assigned")
citi_df_mod["status"] = citi_df_mod["status"].str.replace("CANC","Cancelled")
citi_df_mod["status"] = citi_df_mod["status"].str.replace("FAIL","Failed")
citi_df_mod["status"] = citi_df_mod["status"].str.replace("HOLD","On Hold")

# 7) Rename values in department column
citi_df_mod["department"]  = citi_df_mod["department"].str.replace("IT","Information Technology")
citi_df_mod["department"] = citi_df_mod["department"].str.replace("KCPD","Police")
citi_df_mod["department"] = citi_df_mod["department"].str.replace("Parks & Recreation","Parks & Rec")
citi_df_mod["department"] = citi_df_mod["department"].str.replace("Parks & Rec","Parks & Recreation")
citi_df_mod["department"] = citi_df_mod["department"].str.replace("Parks and Rec","Parks & Recreation")

In [10]:
# 8) Sort the Data by Ascending Month-Day
citi_df_mod = citi_df_mod.sort_values("creation_month-day", ascending=True)
citi_df_mod

,source,department,category,type,detail,year,creation_date,creation_time,status,days_to_close,street_address,zip code,council_district,lat,lon,creation_month-day
1118386,Phone,Public Works,Streets / Roadways / Alleys,Street Clean Up,District 3,2017,01/01/2017,03:21 PM,Resolved,2,9333 Bales Dr,64132.0,5.0,38.955218,-94.546151,01/01
48727,Online,Public Works,Streets / Roadways / Alleys,Pothole,District 2,2020,01/01/2020,01:04 PM,Resolved,5,340 W 5th St,64105.0,4.0,39.107857,-94.587810,01/01
1358819,Online,Public Works,Lights / Signals,Street Light,Blinking,2019,01/01/2019,08:18 AM,Resolved,2,11518 N Virginia Ave,64155.0,2.0,39.302604,-94.562697,01/01
1017350,Online,NHS,Trash / Recycling,Recycling,Missed by Contractor South,2016,01/01/2016,09:49 PM,Resolved,3,707 W 74th St,64114.0,6.0,38.994115,-94.597956,01/01
1122622,Phone,Parks & Recreation,Trees,Storm Damage,Tree Down,2017,01/01/2017,10:36 PM,Resolved,2,4400 JACKSON AVE,64130.0,3.0,39.045664,-94.535984,01/01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010146,Phone,Public Works,Streets / Roadways / Alleys,Street Services,Question,2015,12/31/2015,11:29 AM,Resolved,63,3000 DE GROFF WAY,64108.0,4.0,39.072484,-94.581003,12/31
1010147,Phone,NHS,Trash / Recycling,Recycling,Missed by Contractor North,2015,12/31/2015,08:05 AM,Resolved,2,8531 NE 105th Ter,64157.0,1.0,39.284113,-94.476673,12/31
1010148,Online,Parks & Recreation,Parks & Recreation,Services,Question,2015,12/31/2015,12:57 PM,Resolved,5,101 Southwest Blvd,64108.0,4.0,39.089967,-94.585152,12/31
443970,Phone,NHS,Trash,Contractor,All,2009,12/31/2009,03:50 PM,Resolved,4,7335 WARD PKWY,64114.0,6.0,38.995038,-94.603001,12/31


In [11]:
# Setup connection to mongodb
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

# Select database and collection to use
client.drop_database('citi_data')
db = client.citi_data
calls = db.calls


data = citi_df_mod.to_dict(orient='records')  # Here's our added param..

calls.insert_many(data)
client.close()

In [9]:
citi_df_mod.isnull().sum().sum()

0